In [6]:
import stanfordnlp

In [7]:
nlp_uk = stanfordnlp.Pipeline(lang="uk")

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/dbabenko/stanfordnlp_resources/uk_iu_models/uk_iu_tokenizer.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/dbabenko/stanfordnlp_resources/uk_iu_models/uk_iu_tagger.pt', 'pretrain_path': '/home/dbabenko/stanfordnlp_resources/uk_iu_models/uk_iu.pretrain.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/dbabenko/stanfordnlp_resources/uk_iu_models/uk_iu_lemmatizer.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/dbabenko/stanfordnlp_resources/uk_iu_models/uk_iu_parser.pt', 'pretrain_path': '/home/dbabenko/stanfordnlp_resources/uk_iu_models/uk_iu

In [85]:
import pandas as pd
import json

In [53]:
import warnings
warnings.filterwarnings('ignore')

In [83]:
def save_data_to_json(data, file):
    with open(file, 'w') as f:
        json.dump(data, f)

In [92]:
def sort_dict_by_values(d, reverse=False):
    return {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=reverse)}

### Load data

In [33]:
train_df = pd.read_csv("/home/dbabenko/DS/NLP/Detect-emotion-sentimental/dataset/booking/booking-train.csv")

In [34]:
train_df.head()

,title,pos_text,neg_text,ratingValue,bestRating,hotel,rating
0,Лише дівчата на рецепції - три рази мені мінял...,Лише дівчата на рецепції - три рази мені мінял...,"Все. Одного досвіду вистарчило, щоб більше сюд...",4.6,10.0,verhovina.uk.html,2
1,"Оформлення кімнати хороше, досить приємне, на ...","Оформлення кімнати хороше, досить приємне, на ...",Nan,9.2,10.0,verhovina.uk.html,5
2,"Усе відмінно, завдяки якісному сервісу ми завж...","Усе відмінно, завдяки якісному сервісу ми завж...",Рекомендую людям котрі подорожують власним тра...,10.0,10.0,verhovina.uk.html,5
3,Ціна/якість в принципі,Ціна/якість в принципі,"душова, трохи старий ремонт, але за 500 грн за...",7.5,10.0,verhovina.uk.html,4
4,"Приїхали з сином біля сьомої ранку, сонні та в...","Приїхали з сином біля сьомої ранку, сонні та в...",Nan,8.3,10.0,verhovina.uk.html,4


## Calculate PMI for adjectives abd verbs in existing corpus

In [106]:
import math

### Calculate word occurences in positive and negative reviews

In [70]:
def calc_adj_and_verb_count_dict(texts):
    adjective_count_dict, verb_count_dict = dict(), dict()
    count = 0
    for text in texts:
        if len(text) == 0:
            continue
        if text == "Nan":
            continue
            
        try:
            doc = nlp_uk(text)
            for sent in doc.sentences:
                for word in sent.words:
                    lemma = word.lemma
                    if lemma.isdigit():
                        continue
                    if word.upos == "ADJ":
                        if lemma not in adjective_count_dict:
                            adjective_count_dict[lemma] = 1
                        else:
                            adjective_count_dict[lemma] += 1
                            
                    if word.upos == "VERB":
                        if lemma not in verb_count_dict:
                            verb_count_dict[lemma] = 1
                        else:
                            verb_count_dict[lemma] += 1
            count += 1

        except: 
            continue
            
    return adjective_count_dict, verb_count_dict, count


In [79]:
%time pos_adjective_count_dict, pos_verb_count_dict, pos_count = calc_adj_and_verb_count_dict(train_df['pos_text'].values)

CPU times: user 1h 22min 23s, sys: 11.7 s, total: 1h 22min 34s
Wall time: 1h 22min 38s


In [93]:
pos_adjective_count_dict = sort_dict_by_values(pos_adjective_count_dict, reverse=True)

In [94]:
pos_adjective_count_dict

{'відмінний': 9191,
 'привітний': 8783,
 'зручний': 8572,
 'хороший': 8545,
 'чистий': 8470,
 'смачний': 5505,
 'приємний': 5079,
 'великий': 5056,
 'чудовий': 3958,
 'затишний': 3909,
 'новий': 3358,
 'комфортний': 3070,
 'гарний': 2898,
 'ввічливий': 2796,
 'просторий': 2638,
 'гарне': 2381,
 'чуйний': 2189,
 'прекрасний': 1920,
 'доброзичливий': 1744,
 'необхідне': 1385,
 'розташований': 1243,
 'свіжий': 1239,
 'уважний': 1239,
 'перший': 1202,
 'ідеальний': 1186,
 'красивий': 1108,
 'тихий': 1103,
 'невеликий': 1078,
 'білизний': 1040,
 'задоволений': 1030,
 'величезний': 1029,
 'безкоштовний': 986,
 'маленький': 960,
 'шикарний': 958,
 'сучасний': 942,
 'ресепшений': 930,
 'гарячий': 901,
 'хороше': 885,
 'прекрасне': 877,
 'непоганий': 862,
 'теплий': 790,
 'нормальний': 789,
 'необхідний': 778,
 'ванна': 769,
 'постільний': 752,
 'готовий': 711,
 'затишній': 705,
 'різноманітний': 686,
 'високий': 673,
 'шведський': 648,
 'ванний': 631,
 'окремий': 604,
 'наступний': 574,
 'вищи

In [95]:
pos_verb_count_dict = sort_dict_by_values(pos_verb_count_dict, reverse=True)

In [96]:
pos_verb_count_dict

{'бути': 10911,
 'сподобатися': 6899,
 'зупинятися': 1837,
 'рекомендувати': 1657,
 'відповідати': 1517,
 'знаходитися': 1502,
 'приїхати': 1379,
 'працювати': 1366,
 'немати': 1139,
 'Рекомендувати': 1136,
 'дякувати': 1113,
 'допомогти': 1048,
 'порадувати': 1046,
 'повернутися': 973,
 'залишити': 848,
 'могти': 830,
 'залишитися': 827,
 'хотітися': 687,
 'заселити': 658,
 'зупинитися': 604,
 'йти': 584,
 'знайти': 579,
 'зустріти': 549,
 'хотіти': 543,
 'зробити': 529,
 'радити': 507,
 'їхати': 488,
 'жити': 478,
 'сказати': 467,
 'виходити': 463,
 'відпочивати': 412,
 'дістатися': 402,
 'відпочити': 400,
 'переночувати': 397,
 'дати': 391,
 'дозволити': 387,
 'готувати': 387,
 'замовити': 380,
 'здивувати': 374,
 'входити': 369,
 'мати': 363,
 'прибирати': 360,
 'запропонувати': 355,
 'брати': 351,
 'приїжджати': 348,
 'поселити': 342,
 'дивитися': 340,
 'провести': 328,
 'поїсти': 327,
 'спати': 317,
 'чути': 312,
 'перевершити': 309,
 'надати': 309,
 'добиратися': 300,
 'принести

In [97]:
save_data_to_json(pos_adjective_count_dict, 'data/positve-adjectives.json')
save_data_to_json(pos_verb_count_dict, 'data/positve-verbs.json')

In [80]:
%time neg_adjective_count_dict, neg_verb_count_dict, neg_count = calc_adj_and_verb_count_dict(train_df['neg_text'].values)

CPU times: user 1h 6min 25s, sys: 7.64 s, total: 1h 6min 33s
Wall time: 1h 6min 32s


In [98]:
neg_adjective_count_dict = sort_dict_by_values(neg_adjective_count_dict, reverse=True)

In [99]:
neg_adjective_count_dict

{'маленький': 2872,
 'ванний': 2749,
 'поганий': 2391,
 'перший': 1597,
 'відсутній': 1517,
 'брудний': 1517,
 'старий': 1515,
 'великий': 1329,
 'жахливий': 1307,
 'гарячий': 1235,
 'сусідній': 1214,
 'душова': 1129,
 'ресепшений': 995,
 'другий': 979,
 'холодний': 924,
 'душовий': 856,
 'єдиний': 809,
 'загальний': 769,
 'білизний': 733,
 'постільний': 726,
 'незручний': 726,
 'неприємний': 723,
 'хороший': 719,
 'зручний': 676,
 'чистий': 634,
 'слабкий': 606,
 'невеликий': 599,
 'розташований': 593,
 'зламаний': 558,
 'повинний': 537,
 'новий': 530,
 'ванна': 512,
 'тонкий': 510,
 'окремий': 508,
 'відкритий': 507,
 'додатковий': 482,
 'високий': 474,
 'туалетний': 453,
 'нормальний': 448,
 'повний': 446,
 'наступний': 438,
 'гучний': 417,
 'одноразовий': 416,
 'кращий': 415,
 'теплий': 412,
 'двоспальний': 400,
 'закритий': 386,
 'приємний': 366,
 'вхідний': 359,
 'смачний': 356,
 'попередній': 352,
 'сильний': 351,
 'готовий': 351,
 'останній': 351,
 'включений': 347,
 'платний':

In [100]:
neg_verb_count_dict = sort_dict_by_values(neg_verb_count_dict, reverse=True)

In [101]:
neg_verb_count_dict

{'бути': 9910,
 'немати': 8741,
 'працювати': 3765,
 'сподобатися': 2386,
 'могти': 2266,
 'довестися': 1777,
 'сказати': 1642,
 'чути': 1638,
 'спати': 1581,
 'знайти': 1353,
 'хотітися': 1304,
 'знаходитися': 1231,
 'виходити': 1147,
 'відповідати': 1112,
 'йти': 1112,
 'мати': 1093,
 'виявитися': 1042,
 'чекати': 1006,
 'зробити': 999,
 'вистачати': 853,
 'хотіти': 791,
 'приїхати': 776,
 'доводитися': 765,
 'знати': 755,
 'дати': 745,
 'стояти': 726,
 'ходити': 707,
 'просити': 692,
 'жити': 674,
 'попросити': 674,
 'брати': 649,
 'принести': 614,
 'відкрити': 612,
 'робити': 608,
 'прибирати': 571,
 'запропонувати': 559,
 'заважати': 558,
 'прийти': 553,
 'зупинятися': 548,
 'змогти': 545,
 'вийти': 545,
 'думати': 541,
 'прибрати': 521,
 'поставити': 518,
 'відбуватися': 515,
 'рекомендувати': 515,
 'залишитися': 514,
 'їхати': 502,
 'бронювати': 497,
 'додати': 487,
 'дивитися': 474,
 'закриватися': 470,
 'поміняти': 455,
 'взяти': 454,
 'вирішити': 453,
 'отримати': 450,
 'замо

In [102]:
save_data_to_json(neg_adjective_count_dict, 'data/negative-adjectives.json')
save_data_to_json(neg_verb_count_dict, 'data/negative-verbs.json')

### Pointwise mutual information

The PMI is calculated anallogicaly like here https://github.com/joonilahn/Sentiment-Analysis-for-Hotel-Reviews for unigrams.

In [114]:
def calcultae_pmi(word_count_dict, opposite_word_count_dict, count, opposite_count):
    result = dict()
    all_count = count + opposite_count
    p_x = count / all_count
    for word in word_count_dict:
        
        p_y_num = word_count_dict[word]
        if word in opposite_word_count_dict:
            p_y_num += opposite_word_count_dict[word]
            
        p_y = p_y_num / all_count
        p_xy = word_count_dict[word] / all_count
        
        pmi = math.log10(p_xy / (p_x * p_y))
        
        result[word] = pmi
    
    result = sort_dict_by_values(result, reverse=True)    
    return result
                

In [115]:
pos_adj_pmi_dict = calcultae_pmi(pos_adjective_count_dict, neg_adjective_count_dict, pos_count, neg_count)
neg_adj_pmi_dict = calcultae_pmi(neg_adjective_count_dict, pos_adjective_count_dict, neg_count, pos_count)

In [124]:
save_data_to_json(pos_adj_pmi_dict, 'data/pos-adj-pmi.json')
save_data_to_json(neg_adj_pmi_dict, 'data/neg-adj-pmi.json')

In [116]:
pos_adj_pmi_dict

{'по-домашньому': 0.2524102206336865,
 'недорий': 0.25241022063368646,
 'витриманий': 0.25241022063368646,
 'потьомкінський': 0.25241022063368646,
 'нікелевий': 0.25241022063368646,
 'Адекватн': 0.25241022063368646,
 'круте': 0.25241022063368646,
 'стрийський': 0.25241022063368646,
 'ароматний': 0.25241022063368646,
 'стилізований': 0.25241022063368646,
 'об’єктно': 0.25241022063368646,
 'новизний': 0.25241022063368646,
 'приязний': 0.25241022063368646,
 'удобний': 0.25241022063368646,
 '39;яка': 0.25241022063368646,
 'пречудовий': 0.25241022063368646,
 'неспішний': 0.25241022063368646,
 'комунікабельний': 0.25241022063368646,
 'найкрасивіший': 0.25241022063368646,
 'Лівобережний': 0.25241022063368646,
 'огромнос': 0.25241022063368646,
 'неповторний': 0.25241022063368646,
 'маріїнський': 0.25241022063368646,
 'отчий': 0.25241022063368646,
 'уважній': 0.25241022063368646,
 'університетський': 0.25241022063368646,
 'Милі': 0.25241022063368646,
 'вкусий': 0.25241022063368646,
 'Усміхнений

In [117]:
pos_adj_pmi_dict['привітний']

0.2376325753332973

In [118]:
neg_adj_pmi_dict

{'синтетичний': 0.3557871073638221,
 'похмурий': 0.3557871073638221,
 'голий': 0.3557871073638221,
 'помічений': 0.3557871073638221,
 'розбавлений': 0.3557871073638221,
 'складнощі': 0.3557871073638221,
 'Галасливий': 0.3557871073638221,
 'чуже': 0.3557871073638221,
 'неробочі': 0.3557871073638221,
 'підозрілий': 0.3557871073638221,
 'рван': 0.3557871073638221,
 'придорожній': 0.3557871073638221,
 'настояний': 0.3557871073638221,
 'готівковий': 0.3557871073638221,
 'колосальний': 0.3557871073638221,
 'сирий': 0.3557871073638221,
 'розламаний': 0.3557871073638221,
 'заставлений': 0.3557871073638221,
 'Слабенька': 0.3557871073638221,
 'хиткий': 0.3557871073638221,
 'списаний': 0.3557871073638221,
 'підтверджений': 0.3557871073638221,
 'последний': 0.3557871073638221,
 'гіпсокартонний': 0.3557871073638221,
 'вапняний': 0.3557871073638221,
 'катастрофічний': 0.3557871073638221,
 'пожежна': 0.3557871073638221,
 'неприпустимий': 0.3557871073638221,
 'піонерський': 0.3557871073638221,
 'горіл

In [119]:
pos_verb_pmi_dict = calcultae_pmi(pos_verb_count_dict, neg_verb_count_dict, pos_count, neg_count)
neg_verb_pmi_dict = calcultae_pmi(neg_verb_count_dict, pos_verb_count_dict, neg_count, pos_count)

In [122]:
save_data_to_json(pos_verb_pmi_dict, 'data/pos-verb-pmi.json')
save_data_to_json(neg_verb_pmi_dict, 'data/neg-verb-pmi.json')

In [120]:
pos_verb_pmi_dict

{'хороший': 0.25241022063368646,
 'перевищити': 0.25241022063368646,
 'вкусий': 0.25241022063368646,
 'замечательный': 0.25241022063368646,
 'згладжувати': 0.25241022063368646,
 'зображувати': 0.25241022063368646,
 'заворожувати': 0.25241022063368646,
 'підкуповувати': 0.25241022063368646,
 'здоров&': 0.25241022063368646,
 'Тіхь': 0.25241022063368646,
 'рекомендуватий': 0.25241022063368646,
 'погостювати': 0.25241022063368646,
 'сохранятися': 0.25241022063368646,
 'оближти': 0.25241022063368646,
 'насититися': 0.25241022063368646,
 'спасибі': 0.25241022063368646,
 'вміщувати': 0.25241022063368646,
 'зрівнятися': 0.25241022063368646,
 'віритися': 0.25241022063368646,
 'советует': 0.25241022063368646,
 'пожалувати': 0.25241022063368646,
 'прекрасний': 0.25241022063368646,
 'попіаритися': 0.25241022063368646,
 'підкріпитися': 0.25241022063368646,
 'неперевершити': 0.25241022063368646,
 'задовільняти': 0.25241022063368646,
 'красивля': 0.25241022063368646,
 'посприяти': 0.25241022063368646

In [121]:
neg_verb_pmi_dict

{'стирчити': 0.3557871073638221,
 'зливатися': 0.3557871073638221,
 'продати': 0.3557871073638221,
 'протерт': 0.3557871073638221,
 'аргументувати': 0.3557871073638221,
 'зникати': 0.3557871073638221,
 'тхнь': 0.3557871073638221,
 'обурити': 0.3557871073638221,
 'попадитися': 0.3557871073638221,
 'грюкати': 0.3557871073638221,
 'лити': 0.3557871073638221,
 'вибити': 0.3557871073638221,
 'з’їжджатися': 0.3557871073638221,
 'проголосити': 0.3557871073638221,
 'добавити': 0.3557871073638221,
 'наливати': 0.3557871073638221,
 'утворюватися': 0.3557871073638221,
 'напружити': 0.3557871073638221,
 'густи': 0.3557871073638221,
 'прочистити': 0.3557871073638221,
 'трястися': 0.3557871073638221,
 'пофарбувати': 0.3557871073638221,
 'напрацювати': 0.3557871073638221,
 'призводити': 0.3557871073638221,
 'миритися': 0.3557871073638221,
 'сплакати': 0.3557871073638221,
 'залазити': 0.3557871073638221,
 'допустити': 0.3557871073638221,
 'залетіти': 0.3557871073638221,
 'переконувати': 0.355787107363

As we can see, there are a lot of noises in these dictionaries, more noisy words has less pmi coefficient, so in practice ,ostly of these words will be ignored. For baseline we try this version and will see the result. Later we need to filter this noise.